## Introduction to Linear Programming with Python

### What is [Linear Programming (LP)](https://en.wikipedia.org/wiki/Linear_programming)?

A linear program is a mathematical optimization model that has a linear objective function and a set of linear constraints. It is simple yet a very powerful tool to enable mathematical decision making under constraints. 

- **Understand the problem** :) 
- **Define decision variables**: our choices that are under our control, the unknowns of our problem.
- **Define objective & objective function**: a criterion that we wish to minimize (e.g., cost) or maximize (e.g., profit)
- **Define constraints**: the limitations that restrict our choices for decision variables.

Any LP problem with 2 variables can be solved graphically by plotting the constraint equations and narrowing down the feasible region to find the optimum values of the decision variables..

<img src="images/feasible_region.png" alt="feasible_reg" style="width: 300px;" />

A lot of situations we come across in everyday life have processes that follow linear relationships with several day to day factors. As such, linear programming finds in application in many unique ways and in different domains of science and technology such as routing and logistics, financial planning, manufacturing, product mix etc. 

Here in this notebook we will see an introduction to linear programming, how to use LP framework in Python using [PuLP](https://pythonhosted.org/PuLP/) by giving simple examples and a real life example of optimizing everyday life. 

Once we properly translate the problem with algebraic expressions, we can find solutions to relevant everyday problems. 



In [ ]:
import sys
print("Python version: ", sys.version[:5])

In [ ]:
! pip freeze | grep PuLP
! pip freeze | grep pandas
! pip freeze | grep numpy
! pip freeze | grep seaborn
! pip freeze | grep matplotlib

In [ ]:
# ! pip install pulp
# ! pip install 
# ! pip install unidecode

In [ ]:
from pulp import *
import pandas as pd

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import unidecode
import re

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display

# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)
plt.rcParams['figure.figsize'] = (12, 6)

### Let's start with a simple example: 

You are taking a test in which items of type A are worth 10 points and items of type B are worth 15 points. 
It takes 3 minutes for each item of type A and 6 minutes for each item of type B. 
Total time allowed is 60 minutes, and you may not answer more than 16 questions. 
Assuming all your answers are correct, 
how many items of each type should you answer in order to get the best score?

<img src="images/test.jpg" alt="test" style="width: 900px;" />

#### Solution with using PuLP :

In [ ]:
#define variables
x1 = LpVariable('x1', lowBound=0, cat='Integer') # number of type A questions
x2 = LpVariable('x2', lowBound=0, cat='Integer') # number of type B questions

#objective & objective function
prob = LpProblem("test", LpMaximize) 
prob += 10*x1+15*x2 

#these are constraints and not an objective as there is a equality/inequality
prob += 3*x1+6*x2 <= 60
prob += x1+x2 <= 16

prob.writeLP("test.lp")
print(prob)

In [ ]:
print(prob.variables())
print(prob.objective)
print(prob.constraints)

# print(LpStatus)
print(LpStatus[prob.status])

print(x1.name, "=", x1.varValue)
print(x2.name, "=", x2.varValue)
print(x1.cat, x2.cat)

print(value(prob.objective))

In [ ]:
#solve
prob.solve()

print("Status:", LpStatus[prob.status])

print ("\nIndividual decision_variables: ")
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
print ("\nValue of the obj function: ")
print(value(prob.objective))

### Example 2: Giapetto

<img src="images/giapetto.jpeg" alt="giapetto" style="width: 300px;" />

Giapetto, Inc. manufactures wooden toys and tables for kids. The manufacturer wants to maximize their weekly profit. \\$20 of profit per wooden toy and \\$30 of profit per table. A toy requires 1 hour of finishing labor and 2 hours of carpentry labor. A table requires 2 hours of finishing labor and 1 hour of carpentry labor. 
Each week, Giapetto has only 100 finishing hours and 100 carpentry hours available. 

In [ ]:
x1 = LpVariable("x1", lowBound=0) # Create a variable x1 >= 0
x2 = LpVariable("x2", lowBound=0) # Create another variable x2 >= 0

prob = LpProblem("giapetto", LpMaximize)  # Create a LP maximization problem

prob += 20*x1 + 30*x2  # Objective function

prob += 1*x1 + 2*x2 <= 100  # Finishing hours - constraint #1
prob += 2*x1 + 1*x2 <= 100  # Carpentry hours - constraint #2

prob.writeLP("giapetto.lp")
print(prob)  # Display the LP problem

In [ ]:
#solve
prob.solve()

print("Status:", LpStatus[prob.status])

print ("Individual decision_variables: ")
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
print ("Value of the obj function: ")
print(value(prob.objective))

#### Integer Linear Programming (ILP)

In [ ]:
x1 = LpVariable("x1", lowBound=0, cat='Integer') # Create a variable x1 >= 0
x2 = LpVariable("x2", lowBound=0, cat='Integer') # Create another variable x2 >= 0

prob = LpProblem("giapetto2", LpMaximize)  # Create a LP maximization problem

prob += 20*x1 + 30*x2  # Objective function

prob += 1*x1 + 2*x2 <= 100  # Finishing hours - constraint #1
prob += 2*x1 + 1*x2 <= 100  # Carpentry hours - constraint #2

prob.writeLP("giapetto2.lp")
print(prob) #Display the LP problem

In [ ]:
#solve
prob.solve()

print("Status:", LpStatus[prob.status])

print ("Individual decision_variables: ")
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
print ("Value of the obj function: ")
print(value(prob.objective))

### Example 3: Snack Bar

<img src="images/snack.png" alt="snack" style="width: 200px;" />

A snack bar cooks and sells hamburgers and hot dogs during football games. To stay in business it must sell at least 10 hamburgers but cannot cook more than 40. It must also sell at least 30 hot dogs but cannot cook more than 70. It cannot cook more than 90 sandwiches all together. The profit on a hamburger is \\$0.33 and \\$0.21 on a hot dog. How many of each kind of sandwich should the stand sell to make the maximum profit?

In [ ]:
#define variables
x = LpVariable('x', lowBound=10, upBound=40, cat='Integer') # number of hamburgers
y = LpVariable('y', lowBound=30, upBound=70, cat='Integer') # number of hotdogs 

#objective & objective function
prob = LpProblem("snack", LpMaximize) 
prob += 0.33*x+0.21*y

#constraints 
# prob += x <= 40
# prob += x >= 10
# prob += x <= 70
# prob += x >= 30
prob += x+y <= 90

prob.writeLP("snack.lp")
print(prob)

In [ ]:
#solve
prob.solve()

print("Status:", LpStatus[prob.status])

print ("Individual decision_variables: ")
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
print ("Value of the obj function: ")
print(value(prob.objective))

### Example 4: Cookies

<img src="images/cookie.gif" alt="snack" style="width: 400px;" />

Sylvee is going to bake some cookies for next Women Who Code event :) 
She will make two different kinds, oatmeal-raisin and
chocolate chip. It will cost \\$1.70 to bake a dozen oatmeal-raisin cookies 
and \\$1.20 per dozen for chocolate chip cookies. The number of chocolate chip cookies must
be at least twice the number of oatmeal-raisin cookies. She
will bake at least six dozen cookies total, with no more
than five dozen chocolate chip.
What is the minimum cost to bake the cookies? 
How many dozens of each kind will she be able to bake for that cost? 

### Find the Recipes! 😋


Here in this section, we will use [Epicurious](https://www.epicurious.com/) recipes dataset that is available on [Kaggle](https://www.kaggle.com/hugodarwood/epirecipes/version/2)
    
<table><tr>
<td> <img src="images/epicurious.png" alt="epi" style="width: 300px;"/> </td>
<td> <img src="images/kaggle-logo.png" alt="kaggle" style="width: 300px;"/> </td>
</tr></table>


I cleaned the dataset a little bit and we will use a subset of that to find recipes that satisfies our macronutrient requirements we define.

**Problem**: We would like to have a x number of 5-star recipes that maximize our protein intake while keeping sodium and calorie intake below a certain level. 

In [ ]:
data = pd.read_csv('data.csv')

In [ ]:
# first 5 rows of data
print("\n First 5 rows: ")
display(data.head())

# size of the dataset
print("Number of rows:", data.shape[0]) 
print("Number of columns: ", data.shape[1])

# checking data types in each column
print("\n Data Types: ")
print(data.dtypes)

# checking missing values across dataset
print("\n Missing Values: ")
print(data.isna().sum())

# summary statistics
print("\n Summary Statistics:")
display(data.describe())

In [ ]:
data.rating.value_counts().sort_index()

Take 5-Star recipes and exclude relatively high sodium and calorie ones to create the candidate recipes to choose from:

In [ ]:
five_star = data[data['rating'] == 5]

print('We have {:,} 5-star recipes'.format(len(five_star)))

In [ ]:
a = pd.qcut(five_star['calories'], [0,.33,.66,1], labels=['low','med','high']).rename('cal_class')
b = pd.qcut(five_star['sodium'], [0,.33,.66,1], labels=['low','med','high']).rename('sod_class')

five_star = five_star.join(a).join(b)

In [ ]:
five_star.head()

In [ ]:
display(five_star.groupby('cal_class').agg([np.mean, np.median]))

fig,ax = plt.subplots(1,2,figsize = (15,4)) 

sns.boxplot(y="cal_class", x="calories", data=five_star, ax=ax[0], palette="Set2");
ax[0].set_title("Boxplot of Calories for 5-Star Recipes");
ax[0].set_xlabel('Calories');

sns.boxplot(y="cal_class", x="protein", data=five_star, ax=ax[1], palette="Set2");
ax[1].set_title("Boxplot of Protein for 5-Star Recipes");
ax[1].set_xlabel('Protein');

In [ ]:
display(five_star.groupby('sod_class').agg([np.mean, np.median]))

fig,ax = plt.subplots(1,2,figsize = (15,4)) 

sns.boxplot(y="sod_class", x="sodium", data=five_star, ax=ax[0], palette="Set2");
ax[0].set_title("Boxplot of Sodium for 5-Star Recipes");
ax[0].set_xlabel('Sodium');

sns.boxplot(y="sod_class", x="protein", data=five_star, ax=ax[1], palette="Set2");
ax[1].set_title("Boxplot of Protein for 5-Star Recipes");
ax[1].set_xlabel('Protein');

In [ ]:
recipe_pool = five_star[(five_star['sod_class'] != 'high') & (five_star['cal_class'] != 'high')]

print('When we exclude high sodium and high calorie ones, there are {:,} 5-star recipes to choose from'.format(len(recipe_pool)))


In [ ]:
recipe_pool.title.value_counts().sort_values(ascending=False).head()

In [ ]:
recipe_pool.head()

Some text manipulation:

In [ ]:
df = recipe_pool.copy()

df['title'] = df['title'].map(lambda x: unidecode.unidecode(x).lower())

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, " ")
    return text

df['title'] = df['title'].apply(remove_punctuations)
df['title'] = df['title'].map(lambda x: re.sub("\s\s+" , " ", x.strip()))

In [ ]:
df.title.value_counts().sort_values(ascending=False).head()

Variable names should be all unique!

In [ ]:
df.drop_duplicates('title' ,inplace=True)

In [ ]:
print(df.shape)
df.head()

### Setting up the LP Problem:

Let's initialize the problem with the objective:

In [ ]:
lp_model = pulp.LpProblem('The Diet Problem', pulp.LpMaximize)

In [ ]:
recipes = df['title'].tolist()
calories = df['calories'].tolist()
protein = df['protein'].tolist()
sodium = df['sodium'].tolist()

In [ ]:
# recipes

Identify variables: 

In [ ]:
# Creates a dictionary of LP variables
# This function creates a dictionary of LP Variables with the specified associated parameters.

x = pulp.LpVariable.dict('x_%s', recipes, lowBound=0, upBound = 1, cat='Binary')

print(type(x))
# print(x)

In [ ]:
# for key,value in x.items():
#         print(value)

In [ ]:
print('Number of variables: ', len(x.values()))

In [ ]:
x.get('breakfast bowl with quinoa and berries')

In [ ]:
cal = dict(zip(recipes, calories))
prot = dict(zip(recipes, protein))
sod = dict(zip(recipes, sodium))

In [ ]:
# prot

In [ ]:
print(cal.get('breakfast bowl with quinoa and berries'))
print(prot.get('breakfast bowl with quinoa and berries'))
print(sod.get('breakfast bowl with quinoa and berries'))

In [ ]:
# add objective function to the model - maximize total protein intake
lp_model += sum([prot[i] * x[i] for i in recipes]) 

In [ ]:
#constraints
lp_model += sum([cal[i]*x[i] for i in recipes]) <= 2000 # keep total cal <= 2000 kcal
lp_model += sum([sod[i]*x[i] for i in recipes]) <= 1500 # keep total sod <= 1500 mg
lp_model += sum([x[i] for i in x.keys()]) <= 4 # return 4 or less recipes 

In [ ]:
print(lp_model)
lp_model.writeLP("find_recipes.lp")

In [ ]:
#solve
lp_model.solve()

print("Status:", LpStatus[lp_model.status])
print ("Individual decision_variables: ")
for v in lp_model.variables():
    print(v.name, "=", v.varValue)

☝️This way is hard to read. Let's put the output into a pandas dataframe:

In [ ]:
variable_name = []
variable_value = []

for v in lp_model.variables():
    variable_name.append(v.name)
    variable_value.append(v.varValue)

df2 = pd.DataFrame({'title': variable_name, 'decision': variable_value})

In [ ]:
display(df2.head())
display(df.head())

In [ ]:
# more text cleaning to merge dataframes 
df2['title'] = df2['title'].str[2:].str.replace('_', ' ').str.strip()

In [ ]:
df2.head()

In [ ]:
m = pd.merge(df2, df, how='left', on='title')

In [ ]:
print(m.shape)
print(df.shape)
print(df2.shape)
m.head()

In [ ]:
m[m['decision'] == 1]

In [ ]:
print("Total Protein Intake: ", m[m['decision'] == 1].protein.sum(), ' grams')
print("Total Calories Intake: ", m[m['decision'] == 1].calories.sum(), ' kcal')
print("Total Sodium Intake: ", m[m['decision'] == 1].sodium.sum(), ' miligrams')

<img src="images/thatsallfolks.gif" alt="all" style="width: 300px;" />

## Keep Calm and Optimize On 🤓 

### Thank you! ❤️